# Submission

In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier

In [61]:
# base feature set with advanced and mean encoded features
df_base_adv_mean = pd.read_csv('input/feature_sets/base_adv_mean.csv')

# reading the test file
features_to_drop = ['status_group','id']
df_test_X = pd.read_csv('input/feature_sets/base_adv_mean_test.csv')

In [62]:
X = df_base_adv_mean.drop(['status_group','id','functional needs repair','non functional'], axis=1)
y = df_base_adv_mean['status_group'].values

In [30]:



# a little adjustment
df_test_X.funder_mean_nf.fillna(0, inplace=True) 
df_test_X.funder_mean_fr.fillna(0, inplace=True) 

######################################################################
###        RANDOM FOREST CLASSIFIER        ###########################
######################################################################

rf_balanced = RandomForestClassifier(n_jobs=-1, 
                                     verbose=1, 
                                     bootstrap=False, 
                                     max_depth=101,
                                     min_samples_leaf=1,
                                     min_samples_split=6,
                                     n_estimators=1000
                                    )

#X = df_base_adv_mean.drop(['status_group','id','functional needs repair','non functional'], axis=1)
#y = df_base_adv_mean['status_group'].values

#rf_balanced.fit(X,y)

######################################################################


# prediction
rf_gs_best_mean_sub_1 = rf_best.predict(df_test_X.drop(features_to_drop, axis=1))

# preparing the file
df_test_X['status_group'] = rf_gs_best_mean_sub_1
df_gs_best_mean_sub_2 = df_test_X[['id','status_group']]

#saving the file
df_gs_best_mean_sub_2.to_csv('submissions/balanced_with_smote.csv', index=False)
del df_gs_best_mean_sub_2

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.0s finished


In [50]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.21, random_state=42)

In [51]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

In [52]:
df_base_adv_mean['status_group'].value_counts()/len(df_base_adv_mean)

functional                 0.543081
non functional             0.384242
functional needs repair    0.072677
Name: status_group, dtype: float64

In [53]:
oversample = SMOTE()
undersample = RandomUnderSampler()
steps = [('os',oversample), ('u', undersample)]
pipeline = Pipeline(steps=steps)
X_train_balanced, y_train_balanced = pipeline.fit_resample(X_train, y_train)

In [55]:
len(y_train_balanced[y_train_balanced=='functional']),\
len(y_train_balanced[y_train_balanced=='non functional']),len(y_train_balanced[y_train_balanced=='functional needs repair'])

(25469, 25469, 25469)

### halving grid search cv - 1 (re-test 2) after applying class imbalance

In [23]:
from datetime import datetime
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score
import joblib
import os

In [57]:
# parameter grid
pgrid = {    
    'max_depth' : [101,151],    
    'min_samples_split' : [3,4],
    'min_samples_leaf' : [2],    
}

# specifying the cv
cv_skf = StratifiedKFold(n_splits=5, random_state=None, shuffle=False)

# specifying the model 
rfgs = RandomForestClassifier(n_jobs=-1, verbose=1)

# keep track of the date and time
dt_string = datetime.now().strftime("%d/%m/%Y %H:%M:%S")


# specify the grid search cv
cv = HalvingGridSearchCV(estimator=rfgs,param_grid=pgrid,cv=cv_skf,n_jobs=-1,verbose=10, scoring='accuracy',random_state=0)

# pring the date and time 
print("date and time =", dt_string)

date and time = 04/09/2021 02:04:37


In [58]:
%%time
search = cv.fit(X_train_balanced,y_train_balanced)

fd is '5'
fd is '5'
fd is '5'
fd is '5'
n_iterations: 2
n_required_iterations: 2
n_possible_iterations: 2
min_resources_: 25469
max_resources_: 76407
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 4
n_resources: 25469
Fitting 5 folds for each of 4 candidates, totalling 20 fits
fd is '5'
fd is '5'
fd is '5'
fd is '5'
fd is '5'
fd is '5'
fd is '5'
fd is '5'
fd is '5'
fd is '5'
fd is '5'
fd is '5'
fd is '5'
fd is '5'
fd is '5'
fd is '5'
fd is '5'
fd is '5'
fd is '5'
fd is '5'
fd is '5'
fd is '5'
fd is '5'
fd is '5'
fd is '5'
fd is '5'
fd is '5'
fd is '5'
fd is '5'
fd is '5'
fd is '5'
fd is '5'
fd is '5'
fd is '5'
fd is '5'
fd is '5'
fd is '5'
fd is '5'
fd is '5'
fd is '5'
fd is '5'
fd is '5'
fd is '5'
fd is '5'
fd is '5'
fd is '5'
fd is '5'
fd is '5'
fd is '5'
fd is '5'
fd is '5'
fd is '5'
fd is '5'
fd is '5'
fd is '5'
fd is '5'
fd is '5'
fd is '5'
fd is '5'
fd is '5'
fd is '5'
fd is '5'
fd is '5'
fd is '5'
fd is '5'
fd is '5'
fd is '5'
fd is '5'
fd is '5'
fd is 

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.5s


Wall time: 1min 5s


[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    3.9s finished


In [46]:
search.best_params_

{'max_depth': 101, 'min_samples_leaf': 2, 'min_samples_split': 4}

In [134]:
from sklearn.metrics import classification_report
from imblearn.ensemble import BalancedRandomForestClassifier

### applying the class weight

In [117]:
def class_weight(labels_dict,mu=0.15):
    total = np.sum(list(labels_dict.values()))
    keys = labels_dict.keys()
    weight = dict()
    for i in keys:
            #score = np.log(mu*total/float(labels_dict[i]))
            #score = total/float(labels_dict[i])
            score = float(labels_dict[i])/total
            weight[i] = score if score > 1 else 1
    return weight

# random labels_dict
labels_dict = df_base_adv_mean['status_group'].value_counts().to_dict()
weights = class_weight(labels_dict)

In [124]:
rf_balanced = RandomForestClassifier(n_jobs=-1, 
                                     verbose=1, 
                                     bootstrap=True, 
                                     max_depth=101,
                                     min_samples_leaf=1,
                                     min_samples_split=6,
                                     n_estimators=1000,
                                     class_weight=weights #'balanced_subsample' #balanced, balanced_subsample
                                    )

In [125]:
rf_best = rf_balanced#search.best_estimator_
rf_best.fit(X_train,y_train)

# get the prediction
rfpred = rf_best.predict(X_test)

# print classification report
cr = classification_report(y_test, rfpred)

print(cr)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:    8.9s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:   16.2s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:   20.7s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:    0.6s
[Parallel(n_jobs=8)]: Done 1000 out of 1000 | elapsed:    0.8s finished


                         precision    recall  f1-score   support

             functional       0.82      0.89      0.85      6790
functional needs repair       0.54      0.35      0.43       885
         non functional       0.84      0.79      0.82      4799

               accuracy                           0.81     12474
              macro avg       0.73      0.68      0.70     12474
           weighted avg       0.81      0.81      0.81     12474



In [147]:
rf_balanced = BalancedRandomForestClassifier(n_jobs=-1, 
                                     verbose=1, 
                                     bootstrap=True, 
                                     max_depth=101,
                                     min_samples_leaf=1,
                                     min_samples_split=6,
                                     n_estimators=1000,
                                     class_weight='balanced_subsample'                                         
                                    )

In [148]:
rf_best = rf_balanced#search.best_estimator_
rf_best.fit(X_train,y_train)

# get the prediction
rfpred = rf_best.predict(X_test)

# print classification report
cr = classification_report(y_test, rfpred)

print(cr)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    4.5s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   10.9s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:   20.6s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:   26.7s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.5s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:    1.1s
[Parallel(n_jobs=8)]: Done 1000 out of 1000 | elapsed:    1.4s finished


                         precision    recall  f1-score   support

             functional       0.85      0.70      0.77      6790
functional needs repair       0.26      0.79      0.39       885
         non functional       0.85      0.74      0.79      4799

               accuracy                           0.72     12474
              macro avg       0.65      0.74      0.65     12474
           weighted avg       0.81      0.72      0.75     12474



In [142]:
df_validation = pd.DataFrame(np.c_[y_test, rfpred], columns=['test','pred'])

In [143]:
df_validation[(df_validation['test']=='functional needs repair') & (df_validation['test']!=df_validation['pred'])].pred.value_counts()

functional        69
non functional    53
Name: pred, dtype: int64

In [144]:
df_validation[(df_validation['test']=='functional needs repair') & (df_validation['test']==df_validation['pred'])].pred.value_counts()

functional needs repair    763
Name: pred, dtype: int64

In [130]:
437+137+311

885

In [133]:
((437+137)*100)/885

64.85875706214689

In [116]:
def class_weight(labels_dict,mu=0.15):
    total = np.sum(list(labels_dict.values()))
    keys = labels_dict.keys()
    weight = dict()
    for i in keys:
            score = float(labels_dict[i])/total
            weight[i] = score if score < 1 else 1
    return weight

# random labels_dict
labels_dict = df_base_adv_mean['status_group'].value_counts().to_dict()
weights = class_weight(labels_dict)
print(weights)

{'functional': 0.543080808080808, 'non functional': 0.3842424242424242, 'functional needs repair': 0.07267676767676767}


In [64]:
labels_dict = df_base_adv_mean['status_group'].value_counts().to_dict()

In [67]:
labels_dict

{'functional': 32259, 'non functional': 22824, 'functional needs repair': 4317}

In [85]:
mu=0.15
total = np.sum(list(labels_dict.values()))
keys = labels_dict.keys()
for i in keys:
    score = np.log(mu*total/float(labels_dict[i]))
    print(score)

-1.286622832593781
-0.9406383724284809
0.7246135249093774


In [87]:
df_base_adv_mean['status_group'].value_counts()/len(df_base_adv_mean)

functional                 0.543081
non functional             0.384242
functional needs repair    0.072677
Name: status_group, dtype: float64

In [82]:
np.sum(list(labels_dict.values()))

59400

In [77]:
np.sum(labels_dict.values())

dict_values([32259, 22824, 4317])

In [83]:
len(df_base_adv_mean)

59400